# 效能指標-預測最低價格

In [2]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# --------------------------
# 讀取與前處理
# --------------------------
file_path = '/Users/yuching/Documents/專題/merge_and_cleaned/final_data/short_flight.csv'
data = pd.read_csv(file_path)

economy_class_data = data[data["艙等"] == "經濟艙"].copy()
business_class_data = data[data["艙等"] == "商務艙"].copy()

categorical_cols = ['星期', '出發時段', '出發機場代號', '抵達時段', '抵達機場代號',
                    '航空公司', '航空聯盟', '機型分類', '是否過夜', '是否為平日', '機型', '假期', 'Region']
economy_class_data = pd.get_dummies(economy_class_data, columns=categorical_cols, drop_first=True)
business_class_data = pd.get_dummies(business_class_data, columns=categorical_cols, drop_first=True)

common_cols = list(set(economy_class_data.columns) & set(business_class_data.columns))
economy_class_data = economy_class_data[common_cols]
business_class_data = business_class_data[common_cols]

num_cols = ['飛行時間_分鐘', '經濟指標', '機場指標', 'competing_flights']
scaler = StandardScaler()
economy_class_data[num_cols] = scaler.fit_transform(economy_class_data[num_cols])
business_class_data[num_cols] = scaler.transform(business_class_data[num_cols])

target_keywords = ['出發時段_', '出發機場代號_', '抵達時段_', '航空聯盟_', '機型分類_', '假期_', '是否為平日_']
econ_dummy = [c for c in economy_class_data.columns if any(k in c for k in target_keywords)]
biz_dummy  = [c for c in business_class_data.columns if any(k in c for k in target_keywords)]
economy_feature_cols = econ_dummy + num_cols
business_feature_cols = biz_dummy + num_cols

# --------------------------
# 最佳參數 (short_最低價格_log)
# --------------------------
best_params_econ = {
    'RandomForest': {'n_estimators':250,'max_depth':30,'min_samples_split':9,'min_samples_leaf':3},
    'SVR':          {'C':6.0,'epsilon':0.1,'gamma':'scale'},
    'XGBoost':      {'n_estimators':250,'learning_rate':0.05,'max_depth':6,'subsample':0.8,'colsample_bytree':0.8}
}
best_params_biz = {
    'RandomForest': {'n_estimators':200,'max_depth':30,'min_samples_split':9,'min_samples_leaf':4},
    'SVR':          {'C':5.0,'epsilon':0.1,'gamma':'scale'},
    'XGBoost':      {'n_estimators':200,'learning_rate':0.05,'max_depth':5,'subsample':0.8,'colsample_bytree':0.7}
}

seed = 123

def train_and_predict_model(ModelClass, params, X_train, X_test, y_train, y_test, label):
    # 如果是 XGB，加上 n_jobs、tree_method、verbosity
    if ModelClass is XGBRegressor:
        model = ModelClass(**params, random_state=seed, n_jobs=-1, tree_method='hist', verbosity=0)
    else:
        model = ModelClass(**params, random_state=seed, n_jobs=-1) \
                if ModelClass is RandomForestRegressor \
                else ModelClass(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2  = r2_score(y_test, y_pred)
    print(f"📌 {label} 測試集  MSE: {mse:.4f}  R²: {r2:.4f}")
    return y_pred

# === 經濟艙模型訓練 ===
print("====== 經濟艙模型訓練 ======")
X_e = economy_class_data[economy_feature_cols]
y_e = economy_class_data["最低價格_log"]
X_tr_e, X_te_e, y_tr_e, y_te_e = train_test_split(X_e, y_e, test_size=0.3, random_state=seed)

rf_pred_e   = train_and_predict_model(RandomForestRegressor, best_params_econ['RandomForest'], X_tr_e, X_te_e, y_tr_e, y_te_e, "RandomForest")
svr_pred_e  = train_and_predict_model(SVR, best_params_econ['SVR'], X_tr_e, X_te_e, y_tr_e, y_te_e, "SVR")
xgb_pred_e  = train_and_predict_model(XGBRegressor, best_params_econ['XGBoost'], X_tr_e, X_te_e, y_tr_e, y_te_e, "XGBoost")

economy_results = economy_class_data.loc[X_te_e.index].copy()
economy_results["預測最低價格_rf"]  = np.exp(rf_pred_e)
economy_results["預測最低價格_svr"] = np.exp(svr_pred_e)
economy_results["預測最低價格_xgb"] = np.exp(xgb_pred_e)

# === 商務艙模型訓練 ===
print("\n====== 商務艙模型訓練 ======")
X_b = business_class_data[business_feature_cols]
y_b = business_class_data["最低價格_log"]
X_tr_b, X_te_b, y_tr_b, y_te_b = train_test_split(X_b, y_b, test_size=0.3, random_state=seed)

rf_pred_b   = train_and_predict_model(RandomForestRegressor, best_params_biz['RandomForest'], X_tr_b, X_te_b, y_tr_b, y_te_b, "RandomForest")
svr_pred_b  = train_and_predict_model(SVR, best_params_biz['SVR'], X_tr_b, X_te_b, y_tr_b, y_te_b, "SVR")
xgb_pred_b  = train_and_predict_model(XGBRegressor, best_params_biz['XGBoost'], X_tr_b, X_te_b, y_tr_b, y_te_b, "XGBoost")

business_results = business_class_data.loc[X_te_b.index].copy()
business_results["預測最低價格_rf"]  = np.exp(rf_pred_b)
business_results["預測最低價格_svr"] = np.exp(svr_pred_b)
business_results["預測最低價格_xgb"] = np.exp(xgb_pred_b)

# === 計算並顯示平均節省比例 ===
def calc_eff(df, col):
    return ((df["隨機購買平均價格"] - df[col]) / df["隨機購買平均價格"]).mean() * 100

print("\n====== 平均節省比例 ======")
print(f"經濟艙 RF  : {calc_eff(economy_results, '預測最低價格_rf'):.2f}%")
print(f"經濟艙 SVR : {calc_eff(economy_results, '預測最低價格_svr'):.2f}%")
print(f"經濟艙 XGB : {calc_eff(economy_results, '預測最低價格_xgb'):.2f}%")
print(f"商務艙 RF  : {calc_eff(business_results, '預測最低價格_rf'):.2f}%")
print(f"商務艙 SVR : {calc_eff(business_results, '預測最低價格_svr'):.2f}%")
print(f"商務艙 XGB : {calc_eff(business_results, '預測最低價格_xgb'):.2f}%")

====== 經濟艙模型訓練 ======
📌 RandomForest 測試集  MSE: 0.0441  R²: 0.8547
📌 SVR 測試集  MSE: 0.0493  R²: 0.8378
📌 XGBoost 測試集  MSE: 0.0413  R²: 0.8641

====== 商務艙模型訓練 ======
📌 RandomForest 測試集  MSE: 0.0125  R²: 0.8457
📌 SVR 測試集  MSE: 0.0141  R²: 0.8260
📌 XGBoost 測試集  MSE: 0.0121  R²: 0.8510

====== 平均節省比例 ======
經濟艙 RF  : 19.23%
經濟艙 SVR : 19.13%
經濟艙 XGB : 19.39%
商務艙 RF  : 8.55%
商務艙 SVR : 7.52%
商務艙 XGB : 8.54%
